In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, chisquare
from math import pi, cos
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM
from math import exp, pi, cos  # Make sure exp is imported here
import tensorflow as tf

phase = np.random.rand() * 2 * pi
PtBatMax, PtMaxPanel = 43.68, 24.24
K, Eff = 191.1, 21.528 / 100
TInitial, TFinal, Dt = 0.0, 97.0, 1.0
SocInitial, HardDeck, SoftDeck = 95, 20, 30
NomConsum = 2
VOLTAGE = 12.0

TasksDf = pd.DataFrame({
    'TaskID': [1, 2, 3, 4],
    'Power': [150, 2.0, 1.0, 2.5],
    'StartTime': [10, 30, 50, 70],
    'Duration': [5, 1, 5, 1],
    'PriorityTransmission': [0.2, 1, 0.4, 0.3],
    'PriorityExecution': [0.2, 1, 0.1, 0.9]
})

# Define the dispersion (σ) as a constant for now
sigma = 1  # Adjust this as necessary

# Define the g(k) function
def g(k):
    if k == 1:
        return 1
    elif k == 2:
        return 0.5
    elif k == 3:
        return 0.25
    else:
        return 0

def build_model(input_dim):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output for binary decision

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_state_vector(task, times, powers, power_derivs, power_derivs2, socs, soc_derivs, soc_derivs2):
    # Combine task details with current system metrics
    task_details = np.array([task['Power'], task['StartTime'], task['Duration'], task['PriorityTransmission'], task['PriorityExecution']])
    current_power_metrics = np.array([powers, power_derivs, power_derivs2])
    soc_metrics = np.array([socs, soc_derivs, soc_derivs2])
    
    # Flatten the arrays to form a single state vector
    state_vector = np.concatenate((task_details, current_power_metrics.flatten(), soc_metrics.flatten()))
    return state_vector

# Define the reward function for a single task
def task_reward(task, t_required, t_actual, sigma, d, k):
    s_j = 1 if task['TaskID'] in historical_task_status else 0
    p_j = task['PriorityExecution']
    t_f = t_actual - t_required
    gaussian_penalty = exp(-(t_f ** 2) / (2 * sigma ** 2))
    return s_j * p_j * gaussian_penalty * d * g(k)

# Define the overall reward function
def total_reward(tasks_df, historical_task_status, socs, E_max, E_L, alpha, sigma, k):
    Y_j_total = sum(task_reward(task, task['StartTime'], task['Duration'], sigma, 1, k) for _, task in tasks_df.iterrows())
    soc_penalty = alpha * sum(((E_i - E_L) / (E_max - E_L)) for E_i in socs)
    return Y_j_total - soc_penalty

def available_power(t, phase, k, efficiency, pt_max_panel):
    return max(pt_max_panel * cos(2 * pi * t / TFinal + phase) * efficiency * k, 0)

def execute_tasks(times, tasks_df, model, powers, power_derivs, power_derivs2, socs, soc_derivs, soc_derivs2):
    consumption = np.zeros(len(times))
    task_active = np.zeros(len(times), dtype=bool)
    historical_task_status = np.zeros((len(times), len(tasks_df)), dtype=int)  # Track task execution history

    for index, task in tasks_df.iterrows():
        start_index = np.argmax(times >= task['StartTime'])
        end_index = np.argmax(times >= (task['StartTime'] + task['Duration']))

        if start_index and end_index:
            state_vector = create_state_vector(task, times[start_index:end_index+1], powers[start_index:end_index+1], power_derivs[start_index:end_index+1], power_derivs2[start_index:end_index+1], socs[start_index:end_index+1], soc_derivs[start_index:end_index+1], soc_derivs2[start_index:end_index+1])
            decision = model.predict(np.array([state_vector])) > 0.5  # Using a threshold of 0.5 for binary decision

            if decision:
                consumption[start_index:end_index + 1] += task['Power'] / (end_index - start_index + 1)
                task_active[start_index:end_index + 1] = True
                historical_task_status[start_index:end_index + 1, index] = 1  # Mark as active during the task duration

    # Preserve historical status of tasks that have been executed
    for t in range(1, len(times)):
        historical_task_status[t] |= historical_task_status[t-1]
        
    return consumption, historical_task_status, task_active, data_generated

def is_task_running(times, task_active):
    for t, active in zip(times, task_active):
        if active:
            print(f"At minute {t}, a task is running.")
        else:
            print(f"At minute {t}, no task is running.")

def manage_soc(times, power, consumption, soc_initial, pt_bat_max, dt, nom_consum):
    soc = np.zeros(len(times))
    soc[0] = soc_initial
    for i in range(1, len(times)):
        energy_generated = power[i - 1] * dt / 60
        energy_consumed = (nom_consum + consumption[i - 1]) * dt / 60
        soc[i] = soc[i - 1] + (energy_generated - energy_consumed) * 100 / pt_bat_max
        soc[i] = max(0, min(100, soc[i]))
    return soc

def calculate_derivatives(data, dt):
    d1 = np.zeros(len(data))
    d2 = np.zeros(len(data))
    for i in range(2, len(data)):
        d1[i] = (3 * data[i] - 4 * data[i - 1] + data[i - 2]) / (2 * dt)
        if i > 2:
            d2[i] = (2 * data[i] - 5 * data[i - 1] + 4 * data[i - 2] - data[i - 3]) / dt**2
    return d1[-1], d2[-1]  # Return only the last elements of the derivative arrays


def sample_parameters(n):
    sampled_params = []
    for _ in range(n):
        k_sampled = norm.rvs(191.1, 5)
        eff_sampled = norm.rvs(0.21528, 0.01)
        sampled_params.append((k_sampled, eff_sampled))
    return sampled_params

def analyze_time_in_soc_ranges(socs, dt):
    ranges = np.arange(0, 110, 10)
    time_in_ranges = {}
    for i in range(len(ranges) - 1):
        range_key = f"{ranges[i]}%-{ranges[i+1]}%"
        time_in_ranges[range_key] = 0

    for soc in socs:
        for i in range(len(ranges) - 1):
            if ranges[i] <= soc < ranges[i+1]:
                range_key = f"{ranges[i]}%-{ranges[i+1]}%"
                time_in_ranges[range_key] += dt
                break

        if soc == 100:
            range_key = f"{ranges[-2]}%-{ranges[-1]}%"
            time_in_ranges[range_key] += dt

    return time_in_ranges

def evaluate_performance(time_in_ranges):
    performance_score = 0
    for range_key, time in time_in_ranges.items():
        if "20%-30%" in range_key:
            performance_score += time * 2
        elif "0%-20%" in range_key:
            performance_score += time * 5
        else:
            performance_score += time
    return performance_score

def print_historical_task_status(times, task_status):
    for time, status in zip(times, task_status):
        print(f"At minute {time}: Historical Task Status = {status}")

# Define the size of the state vector
input_dimension = 30 
nn_model = build_model(input_dimension)
# Initialize the simulation parameters
times = np.arange(TInitial, TFinal + Dt, Dt)
powers = np.zeros_like(times)
# Initialize arrays for derivatives
power_derivs = np.zeros(len(times))
power_derivs2 = np.zeros(len(times))
soc_derivs = np.zeros(len(times))
soc_derivs2 = np.zeros(len(times))
consumption = np.zeros_like(times)
consumption_derivs = np.zeros(len(times))
consumption_derivs2 = np.zeros(len(times))
socs = np.zeros_like(times)
task_active = np.zeros_like(times, dtype=bool)
historical_task_status = np.zeros((len(times), len(TasksDf)), dtype=int)
data_generated = np.zeros(len(TasksDf))  # Assuming data generation needs to be tracked

# Calculate power for the initial condition
powers = np.array([available_power(t, phase, K, Eff, PtMaxPanel) for t in times])

# Loop to simulate the task execution and power management iteratively
for i in range(len(times)):
    
    powers[i] = available_power(times[i], phase, K, Eff, PtMaxPanel)
 
   
    if i == 0:
        consumption[i], temp_hist, task_active[i], data_generated = execute_tasks([times[i]], TasksDf, nn_model, powers[i], 0, 0, SocInitial, 0, 0)

    else:
        consumption[i], temp_hist, task_active[i], data_generated = execute_tasks([times[i]], TasksDf, nn_model, powers[i], power_derivs[i-1], power_derivs2[i-1], socs[i-1], soc_derivs[i-1], soc_derivs2[i-1])


    historical_task_status[i] = temp_hist[0]  # Update historical status

    # Calculate SoC based on current and previous states
    socs[i] = manage_soc([times[i]], powers[i:i+1], consumption[i:i+1], SocInitial if i == 0 else socs[i-1], PtBatMax, Dt, NomConsum)[0]

    # Derivatives might need to be calculated less frequently or can be approximated based on fewer points
    if i >= 2:  # Ensuring there are enough points to calculate derivatives
        power_derivs[i], power_derivs2[i] = calculate_derivatives(powers[:i+1], Dt)[-2:]
        soc_derivs[i], soc_derivs2[i] = calculate_derivatives(socs[:i+1], Dt)[-2:]
        consumption_derivs[i], consumption_derivs2[i] = calculate_derivatives(consumption[:i+1], Dt)[-2:]

# Post-loop: you can perform any final calculations or analytics
is_task_running(times, task_active)

plt.figure(figsize=(18, 13.5))

ax1 = plt.subplot(3, 1, 1)
p1, = ax1.plot(times, powers, label='Power', color='blue')
ax1.set_xlabel('Time (min)')
ax1.set_ylabel('Power (W)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

ax1b = ax1.twinx()
p2, = ax1b.plot(times, power_derivs, label='Power Deriv', linestyle='--', color='lightblue')
p3, = ax1b.plot(times, power_derivs2, label='Power Deriv2', linestyle=':', color='navy')
ax1b.set_ylabel('Power Derivatives (W/min)', color='navy')
ax1b.tick_params(axis='y', labelcolor='navy')
ax1.legend(handles=[p1, p2, p3], loc='upper right')

ax2 = plt.subplot(3, 1, 2)
s1, = ax2.plot(times, socs, label='SoC', color='green')
ax2.set_xlabel('Time (min)')
ax2.set_ylabel('SoC', color='green')
ax2.tick_params(axis='y', labelcolor='green')
ax2.grid(True)

ax2b = ax2.twinx()
s2, = ax2b.plot(times, soc_derivs, label='SoC Deriv', linestyle='--', color='lightgreen')
s3, = ax2b.plot(times, soc_derivs2, label='SoC Deriv2', linestyle=':', color='darkgreen')
ax2b.set_ylabel('SoC Derivatives', color='darkgreen')
ax2b.tick_params(axis='y', labelcolor='darkgreen')
ax2.legend(handles=[s1, s2, s3], loc='upper right')

ax3 = plt.subplot(3, 1, 3)
c1, = ax3.plot(times, consumption, label='Consumption', color='red')
ax3.set_xlabel('Time (min)')
ax3.set_ylabel('Power Consumption (W)', color='red')
ax3.tick_params(axis='y', labelcolor='red')
ax3.grid(True)

ax3b = ax3.twinx()
c2, = ax3b.plot(times, consumption_derivs, label='Consumption Deriv', linestyle='--', color='pink')
c3, = ax3b.plot(times, consumption_derivs2, label='Consumption Deriv2', linestyle=':', color='darkred')
ax3b.set_ylabel('Consumption Derivatives (W/min)', color='darkred')
ax3b.tick_params(axis='y', labelcolor='darkred')
ax3.legend(handles=[c1, c2, c3], loc='upper right')

plt.tight_layout()
plt.savefig('final_dynamic_plot.png')
plt.show()

time_in_ranges = analyze_time_in_soc_ranges(socs, Dt)
performance_score = 100 * evaluate_performance(time_in_ranges)/(TFinal + 1)
print("Time in each SoC range:", time_in_ranges)
print("Performance score:", performance_score)
tasks_vector = np.hstack((TasksDf['Power'].values, TasksDf['StartTime'].values, TasksDf['Duration'].values, TasksDf['PriorityTransmission'].values, TasksDf['PriorityExecution'].values))
neural_network_inputs = np.hstack((powers, power_derivs, power_derivs2, socs, soc_derivs, soc_derivs2), dtype=object)
state_vector = np.hstack((neural_network_inputs, tasks_vector))
print("State Vector:", state_vector)
print("Datos generados por cada tarea (KB):", data_generated)
print_historical_task_status(times, historical_task_status)

alpha = 0.5  # Example value for the weight of SOC goodness
k = 2  # Example value for the number of days since regen

Z = total_reward(TasksDf, historical_task_status, socs, PtBatMax, HardDeck, alpha, sigma, k)

print("Total Reward:", Z)

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates